In [1]:
import os
import re
import random
import gensim
import numpy as np
import pandas as pd
import nltk
from sklearn import cluster
from sklearn import metrics
from sklearn.cluster import SpectralClustering
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from six import iteritems
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn import metrics

C:\Users\ifue3702\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from Litho.nlp_funcs import *
from Litho.similarity import (check_similarity, match_lithcode, jaccard_similarity, 
                              calc_similarity_score, print_sim_compare, merge_similar_words)


C:\Users\ifue3702\AppData\Local\Continuum\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
root = os.getcwd()
paperFolder = 'papers'
stopa = stopwords.words('english')

In [4]:
def parsingPapers(path, text0, stopwords):
    ##returns a list of lists, with words in it.
    with open(os.path.join(path,text0)) as txt:
        data = txt.read()

        # data = re.sub('\\w+\\set\\sal', '<UNK>', data)
        data = re.sub('\\w+\\sand\\s+\\w+\s\\(\\d{4}\\)', '<UNK>', data)
        data = re.sub('[A-Z][a-z]*\s+\\(\\d{4}\\)', '<UNK>', data)
        data = re.sub('\\([^)]*\\)', '<UNK>', data)
        data = re.sub('[A-Z][a-z]*\\set\\sal.', '<UNK>', data)
        data = re.sub('\s+[.]', '.', data)
        data = re.sub('\s+[,]', ',', data)
        data = data.replace(u'\xa0', u' ')
        data = data.split('\\n')
        data = [n.strip().split('.') for n in data]
        data = [[z.strip().split(' ') for z in n] for n in data]
        data = [[z for z in n if len(z) > 5] for n in data if len(n) > 0]
        data = [n for n in data if len(n) > 0]
        data = [n for n in [x for y in data for x in y]]
        data = [[n.lower() for n in l if not re.search('\\d', n)] for l in data]
        data = [[n.replace(',','') for n in l] for l in data]
        data = [[n for n in l if n.isalpha()] for l in data] ####TODO check
        data = [[n for n in l if n not in stopwords] for l in data]
        data = [[n for n in l if '<unk>' not in n] for l in data]
        data = [[n for n in l if len(n)>2] for l in data]
#         data = [[n.replace("'s",'') for n in l] for l in data]
        # data = [[n.split('-') for n in l] for l in data]
        # data = [[n for n in l if not re.search('\\', n)] for l in data]
        txt.close()
    return data


def labeledSentences(tokenizedDescriptions, labels = None):
    sentences = [gensim.models.doc2vec.TaggedDocument(words = i, tags = [labels[n]]) for n, i in enumerate(tokenizedDescriptions)]
    return sentences

In [5]:
papers = [n for n in os.listdir(os.path.join(root, paperFolder)) if n.endswith('.txt')]
papers = np.random.choice(papers, 10000, replace = False).tolist()
trainPapers = [parsingPapers(os.path.join(root,paperFolder), n, stopa) for n in papers]
trainPapers = [n for sublist in trainPapers for n in sublist]

In [6]:
len(trainPapers)

905254

In [7]:
dictionary = gensim.corpora.Dictionary(trainPapers)

In [9]:
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]

In [12]:
for n in once_ids[0:100]:
    print(dictionary[n])

flevoland
rijp
almere
stichtse
kant
swifterbant
disassembled
geovalidation
khataba
geovalidate
roja
dolomiteous
demonte
belbo
orba
stura
scheck
subpanels
generously
lindsleyite
lithos
anis
kubub
hanaus
tschermaks
letlhakane
genotypes
contests
onu
valentina
timbetes
renz
ichnocoenose
euroharp
nica
autocalibrated
funen
redoxfront
lanpingensis
tectochara
subelongta
gyrogona
obtusochara
qianjiangica
spinosalata
sinocypris
yunlongensis
exselsa
likiangensis
eoentelodon
lunania
baoxingsi
yongsheng
tongdian
vallenar
totoralillo
biese
astrocoenia
hexamera
prophyritic
ocoitas
studer
agria
blumebachi
parahopilites
mediterraneotrigonia
domeykanus
parancyloceras
curvimetric
rotomakariri
patiti
ctdo
antimonate
thioantimony
skuhl
mugharan
mochi
tossing
kindled
conscientious
glowing
tungurahua
ngauruhoe
citric
mammillated
hallsworth
mange
onjeongri
pukyong
asrmari
aknowledge
sabins
gregg
aleks
kalinowski
battock
atholl
tummel
muick
portsoy


In [13]:
dictionary = gensim.corpora.Dictionary(trainPapers)
## newt two lines remove tokens contained just once in the dictionary
once_ids = [tokenid for tokenid, docfreq in iteritems(dictionary.dfs) if docfreq == 1]
dictionary.filter_tokens(once_ids)

dictionary.compactify()
print(dictionary)
corpus = [dictionary.doc2bow(text) for text in trainPapers]
print(len(corpus))

Dictionary(80570 unique tokens: ['bradelle', 'mcd', 'mimics', 'investigaciones', 'tightened']...)
905254


In [26]:
#### Developing Model
# trainList = np.random.choice(os.listdir(os.path.join(root,paperFolder)), 1000, replace = False).tolist()
# trainPapers = [parsingPapers(root+paperFolder, n, stopw) for n in trainList]

##labels
labels = list(range(np.sum([len(n) for n in trainPapers])))
labels = ['lab_{}'.format(n) for n in labels]
len(labels)

##sentences
sentences = []
for n in trainPapers:
    for m in n:
        sentences.append(m)
len(sentences)


## Doc2Vec Model
labSen = labeledSentences(sentences, labels = labels)

model = gensim.models.Doc2Vec(size = 300,
                              window = 3,
#                               sample = 1e-3,
                              min_count = 1,
                              workers = 4,
#                               seed = 1,
#                               dm_concat = 1,
#                               dm_mean = 1,
#                               dm = 1)
                              iter = 5)
model.build_vocab(labSen)

In [27]:
#training of model
for epoch in range(5):
    print('iteration '+str(epoch+1))
    model.train(labSen, total_examples=len(labSen), epochs = 5)
    model.alpha -= 0.002
    model.min_alpha = model.alpha

iteration 1
iteration 2
iteration 3
iteration 4
iteration 5


In [4]:
#### Developing Model
# trainList = np.random.choice(os.listdir(root+paperFolder), 1000, replace = False).tolist()
trainList = os.listdir(root+paperFolder)
# trainPapers = [clean(root+paperFolder, n, stopw) for n in trainList]


In [10]:
stopw2 = ['redish', 'reddish', 'red', 'black', 'blackish', 'brown', 'brownish',
          'blue', 'blueish', 'orange', 'orangeish', 'gray', 'grey', 'grayish',
          'greyish', 'white', 'whiteish', 'purple', 'purpleish', 'yellow',
          'yellowish', 'green', 'greenish', 'light', 'very', 'pink', 'coloured', 'multicoloured',
          'dark', 'color', 'colour', 'hard', 'soft', 'water', 'supply', 'fine', 'coarse',
          'medium', 'bearing', 'pipe', 'sticky', 'tough', 'small', 'stiff', 
          'running', 'streaks', 'nominal', 'bands', 'back', 'slippery', 'loose', 
          'broken', 'fractured', 'surface', 'surface', 'rotten', 'compacted', 'seams',
          'dry', 'wet', 'cemented', 'rock', 'trap', 'ridge', 'large', 'small', 'thin', 'soak',
          'creek', 'ridge', 'clean', 'decomposed', 'band']


stopa= stopa+stopw2  # add the additional stopwords above

In [24]:
#use extend so it's a big flat list of vocab
Namoi = 'reclasified.csv'
Data = pd.read_csv(os.path.join(root,Namoi))
Data = Data.dropna(subset=['Description'])
Descriptions = Data[['Description']]
Descriptions=Descriptions.Description.tolist()
Descriptions = [str(n) for n in Descriptions]
lemm_total = []
token_total = []
for i in Descriptions:
    allwords_stemmed = tokenize_and_lemmatize(i, stopa) #for each item in 'synopses', tokenize/stem
    lemm_total.extend([allwords_stemmed]) #extend the 'totalvocab_stemmed' list

    allwords_tokenized = tokenize_only(i, stopa)
    token_total.extend([allwords_tokenized])

print(np.shape(token_total),  np.shape(lemm_total))
Data['words'] = [len(n) for n in token_total]
Data['Tokens'] = token_total
Data['Lemmas'] = lemm_total
Data = Data[Data.words > 0]

(127930,) (127930,)


In [28]:
vectors = []
[vectors.append(np.reshape(model.infer_vector(n), (1,300))) for n in Data.Tokens]
X = np.vstack(vectors)
kmeans = cluster.MiniBatchKMeans(n_clusters = len(Data.second.unique())).fit(X)
pred_classes = kmeans.predict(X)
print(metrics.adjusted_rand_score(Data['second'].tolist(), pred_classes.tolist()),
     metrics.adjusted_mutual_info_score(Data['second'].tolist(), pred_classes.tolist()),
     metrics.completeness_score(Data['second'].tolist(), pred_classes.tolist()))

0.443677201091 0.6660579445 0.674838016866


In [139]:
DataSampled = Data.sample(1000)
DataDes=DataSampled.Description.tolist()
DataDes = [str(n) for n in DataDes]


# #use extend so it's a big flat list of vocab
lemmatized = []
tokenized = []
for i in DataDes:
    allwords_stemmed = tokenize_and_lemmatize(i, stopa) #for each item in 'synopses', tokenize/stem
    lemmatized.extend([allwords_stemmed]) #extend the 'totalvocab_stemmed' list

    allwords_tokenized = tokenize_only(i, stopa)
    tokenized.extend([allwords_tokenized])

print(np.shape(tokenized),  np.shape(lemmatized))

(1000,) (1000,)


In [140]:
lsi = gensim.models.LsiModel(corpus, id2word=dictionary, num_topics=300)

In [279]:
vectors = []
for n, i in enumerate(Data['Tokens'].tolist()):
    vec_bow = dictionary.doc2bow(i)
    vec_lsi = lsi[vec_bow]
    vector = gensim.matutils.sparse2full(vec_lsi, length = 300)
    vectors.append(vector)

In [281]:
Data['vecs'] = vectors
X = np.vstack(Data.vecs)
kmeans = MiniBatchKMeans(n_clusters = len(Data.second.unique()), batch_size = 100).fit(X)
pred_classes = kmeans.predict(X)

print(metrics.adjusted_rand_score(Data.second.tolist(), pred_classes.tolist()),
      metrics.adjusted_mutual_info_score(Data.second.tolist(), pred_classes.tolist()), 
      metrics.completeness_score(Data.second.tolist(), pred_classes.tolist()))

0.649789512228 0.673252481327 0.673567044358


In [278]:
%time tfidf = gensim.models.TfidfModel(corpus, normalize = True)
corpus_tfidf = tfidf[corpus]

Wall time: 35 s


In [292]:
vectors = []
for n, i in enumerate(Data['Tokens'].tolist()):
    vec_bow = dictionary.doc2bow(i)
    vec_lsi = tfidf[vec_bow]
    vector = gensim.matutils.any2sparse(vec_lsi)
    vectors.append(vec_lsi)

In [291]:
Data['vecs'] = vectors
X = np.vstack(Data.vecs)
kmeans = MiniBatchKMeans(n_clusters = len(Data.second.unique()), batch_size = 100).fit(X)
pred_classes = kmeans.predict(X)

print(metrics.adjusted_rand_score(Data.second.tolist(), pred_classes.tolist()),
      metrics.adjusted_mutual_info_score(Data.second.tolist(), pred_classes.tolist()), 
      metrics.completeness_score(Data.second.tolist(), pred_classes.tolist()))

ValueError: all the input array dimensions except for the concatenation axis must match exactly

In [294]:
lda = gensim.models.LdaMulticore(corpus_tfidf, id2word=dictionary, num_topics=50, chunksize=10000, passes=2, workers = 4)

In [304]:
vectors = []
for n, i in enumerate(Data['Tokens'].tolist()):
    vec_bow = dictionary.doc2bow(i)
    vec_lsi = lda[vec_bow]
    vector = gensim.matutils.sparse2full(vec_lsi, length = 50)
    vectors.append(vectors)

In [ ]:
Data['vecs'] = vectors
X = np.vstack(Data.vecs)
kmeans = MiniBatchKMeans(n_clusters = len(Data.second.unique()), batch_size = 100).fit(X)
pred_classes = kmeans.predict(X)

print(metrics.adjusted_rand_score(Data.second.tolist(), pred_classes.tolist()),
      metrics.adjusted_mutual_info_score(Data.second.tolist(), pred_classes.tolist()), 
      metrics.completeness_score(Data.second.tolist(), pred_classes.tolist()))